In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127439")
exp = Experiment(workspace=ws, name="bank-marketing-hyperdrive-2")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127439
Azure region: southcentralus
Subscription id: 8e713106-916f-4177-890e-435b90d7adc4
Resource group: aml-quickstarts-127439


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "simba-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Use existing cluster.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Use existing cluster.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os
import shutil

### YOUR CODE HERE ###
# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.001, 1.0),
    "--max_iter": choice(50, 100, 200)
})

### YOUR CODE HERE ###
# Specify a Policy for early stopping
es_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

### YOUR CODE HERE ###
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',entry_script='train.py',compute_target=cpu_cluster)

### YOUR CODE HERE ###
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=es_policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the RunDetails widget.

### YOUR CODE HERE ###
hdr = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_67eda737-0a71-4aa7-be95-5da5b284f23c
Web View: https://ml.azure.com/experiments/bank-marketing-hyperdrive-2/runs/HD_67eda737-0a71-4aa7-be95-5da5b284f23c?wsid=/subscriptions/8e713106-916f-4177-890e-435b90d7adc4/resourcegroups/aml-quickstarts-127439/workspaces/quick-starts-ws-127439

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-22T23:58:15.253487][API][INFO]Experiment created<END>\n""<START>[2020-11-22T23:58:16.111106][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-22T23:58:16.421534][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-22T23:58:16.5571288Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
# Get your best run and save the model from that run. Use the .get_best_run_by_primary_metric() method of the run to select
# the best hyperparameters for your model

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()
print('Best Run Id: ', best_run.id)
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

best_run.download_file('outputs/hd-model.joblib','./outputs/hd-model.joblib')
model = best_run.register_model(model_name='hd-model',model_path='outputs/hd-model.joblib', tags={'Method':'Hyperdrive'}, properties={'Accuracy': best_run_metrics['Accuracy']})

cpu_cluster.delete()

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split data into train and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=36)

import pandas as pd
training_data = pd.concat((x_train,y_train),axis=1)

from azureml.core import Dataset
train_data = Dataset.from_pandas_dataframe(training_data)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

### YOUR CODE HERE ###
cpu_cluster_name = "jacob-cluster"

try:
    cpu_cluster_automl = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Use existing cluster.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster_automl = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster_automl.wait_for_completion(show_output=True)

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    compute_target=cpu_cluster_automl,
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=2,
    max_concurrent_iterations=4)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment

experiment = Experiment(ws, "bank-marketing-automl")
run = experiment.submit(config=automl_config, show_output=True)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = run.get_output()
print('Best Run Id: ', best_run.id)
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

model = best_run.register_model(model_name='bank-marketing-automl-model',model_path='outputs/bank-marketing-automl-model.pkl', tags={'Method':'AutoML'}, properties={'Accuracy': best_run_metrics['Accuracy']})
print(model)

cpu_cluster_automl.delete()